In [ ]:
!pip install pandas

In [3]:
import pandas as pd

def leggi_e_salva_file(input_file, output_file, nrows):
    # Leggi il file di input con specifica del delimitatore usando regex
    df = pd.read_csv(input_file, nrows=nrows, sep='\t', engine='python', quoting=3)

    # Salva i primi "nrows" campi nel nuovo file
    df.to_csv(output_file, index=False)



In [4]:
def process_data(interactions_file, users_file, tracks_file, artists_file, output_file):
    # Read interaction data into a pandas DataFrame
    interactions_data = pd.read_csv(interactions_file, delimiter=',')

    # Rename columns to remove escape characters
    interactions_data = interactions_data.rename(columns={
        'user_id:token': 'users_id',
        'tracks_id:token': 'tracks_id',
        'timestamp:float': 'timestamp'
    })

    # Calculate the number of interactions for each user-track pair
    user_interaction_counts = interactions_data.groupby(['users_id', 'tracks_id']).size().reset_index().rename(columns={0: 'interaction_count'})

    # Read track data into a pandas DataFrame
    tracks_data = pd.read_csv(tracks_file, sep='\t', names=['tracks_id', 'track_name', 'artist_id'])

    # Merge with track data
    merged_data = pd.merge(user_interaction_counts, tracks_data, on='tracks_id', how='left')

    # Calculate the total number of plays for each track
    track_playcount = interactions_data.groupby('tracks_id')['tracks_id'].count().reset_index(name='total_playcount')

    # Merge the filtered data with the total play count data per track
    final_merged_data = pd.merge(merged_data, track_playcount, on='tracks_id', how='left')

    # Read artist data into a pandas DataFrame
    artists_data = pd.read_csv(artists_file, sep='\t', names=['artist_id', 'artist_name'])

    # Convert 'artist_id' column in artists_data to the same data type as in final_merged_data
    artists_data['artist_id'] = artists_data['artist_id'].astype(final_merged_data['artist_id'].dtype)

    # Merge with artist data
    final_merged_data = pd.merge(final_merged_data, artists_data, on='artist_id', how='left')

    # Select the desired columns
    final_columns = ['users_id', 'tracks_id', 'interaction_count', 'total_playcount', 'track_name', 'artist_name']

    # Save the output to a CSV file
    final_merged_data[final_columns].to_csv(output_file, index=False)


In [5]:
if __name__ == "__main__":
    # Percorso del file delle interazioni utente-traccia
    percorso_file_input = '../LFM-1b/lfm1b-tracks.inter'

    # Chiedi all'utente di inserire il numero di righe da leggere
    inp = input("Inserisci il numero di MILIONI di righe da leggere: ")
    nrows = float(inp) * 1000000

    # Imposta il percorso per il nuovo file
    percorso_file_output = "tracks.csv"

    # Esegui la funzione con il numero specificato di righe
    leggi_e_salva_file(percorso_file_input, percorso_file_output, nrows)
    
    interactions_file = percorso_file_output
    users_file = "../LFM-1b/LFM-1b_users.txt"
    tracks_file = "../LFM-1b/LFM-1b_tracks.txt"
    artists_file = "../LFM-1b/LFM-1b_artists.txt"
    output_file = "sample.csv"

    # Esegui il processo di elaborazione dei dati
    process_data(interactions_file, users_file, tracks_file, artists_file, output_file)


Inserisci il numero di MILIONI di righe da leggere:  1
